In [1]:
import numpy as np
import random
import matplotlib.pyplot as plt
import networkx as nx
from itertools import islice
import itertools
import math
import time
import csv
from math import *
import csv
import os
import sys
from docplex.mp.progress import *
from docplex.mp.progress import SolutionRecorder
import docplex.mp.model as cpx
import networkx as nx
import time
import matplotlib.pyplot as plt


# Path-based exaustive-search scheme 

In [2]:
class Network:
    def __init__(self):
        self.N = 10# number of nodes
        self.L0 = 100 # square dim in km
        self.G = nx.random_geometric_graph(10, 35, dim=2, p=2)
        self.weights = dict()
        self.transmission = dict()
        self.each_path_width = {}
        self.each_repeater_memory = {}
        self.utility_default_value = -100
        self.p={}
        self.repeater_places = []
        self.set_of_paths_default = {}
        self.utility_values = set([])
        self.each_user_pair_dafualt_paths ={}
        self.each_path_e_value ={}
        self.each_path_e2e_F_value={}
        self.each_path_width = {}
        self.path_counter = 0
        self.set_of_paths = {}
        self.multi_path_state = False
        self.set_of_edges = []
        self.num_of_allowed_paths = 1
        
    def generate_random_graph(self):
        # random network graph of N nodes
        
#         self.N = number_of_nodes
        node_list = np.arange(self.N)

        # nodes are distributed uniformly within an L0xL0 square
#         self.L0 = square_dim_in_km
        length = self.L0
        width = self.L0
        dmax = 0.35*self.L0 # cut off for elementary links
        random.seed(33)
        pos = dict()
        for node in range(self.N):
            pos[node] = (random.random()*length,random.random()*width)

        self.G = nx.random_geometric_graph(self.N, dmax, dim=2, pos=pos, p=2)


        for e in self.G.edges():
            dist = np.linalg.norm([pos[e[0]][0]-pos[e[1]][0],pos[e[0]][1]-pos[e[1]][1]])
            self.weights[e] = dist
            self.transmission[e] = 10**(-0.2*dist/10)
            self.transmission[(e[1],e[0])] = 10**(-0.2*dist/10)

        nx.set_edge_attributes(self.G, values = self.weights, name = 'weight')
        nx.set_edge_attributes(self.G, values = self.transmission, name = 'trans')

        # plt.figure(figsize=(4,3))
        # nx.draw(G,pos,with_labels=True)
        # plt.show()
        self.user_pairs = []
        while(len(self.user_pairs)<2):
            random_node1 = random.randint(0,self.N-1)
            random_node2 = random_node1
            while(random_node2==random_node1):
                random_node2 = random.randint(0,self.N-1)
            if (random_node1,random_node2) not in self.user_pairs:
                self.user_pairs.append((random_node1,random_node2))# [(2,8),(5,3)]
        for node1 in self.G.nodes:
            if node not in self.repeater_places:
                self.repeater_places.append(node1)
            for node2 in self.G.nodes:
                if node1!=node2:
                    e = (node1,node2)
                    if e not in self.G.edges:
                        self.G.add_edge(node1,node2,weight=1)
                        shortest_path = self.get_paths_between_user_pairs(e,1)
                        dist = 0
                        for edge in shortest_path:
                            dist = dist+np.linalg.norm([pos[edge[0]][0]-pos[edge[1]][0],pos[edge[0]][1]-pos[edge[1]][1]])
                        self.weights[e] = dist
                        self.transmission[e] = 10**(-0.2*dist/10)
                        self.set_of_edges.append(e)
                        self.set_of_edges.append((e[1],e[0]))
                        self.transmission[(e[1],e[0])] = 10**(-0.2*dist/10)
    def compute_paths_between_pairs(self):
        path_counter = 0
        for pair in self.user_pairs:
            k_paths_user_pair1  = self.get_paths_between_user_pairs(pair,self.number_of_paths)
            first_user_pair_path_ids = []
            for path in k_paths_user_pair1:
                node_indx = 0
                path_edges = []
                for node_indx in range(len(path)-1):                
                    path_edges.append((path[node_indx],path[node_indx+1]))
                    node_indx+=1
                self.set_of_paths_default[path_counter] = path_edges
                first_user_pair_path_ids.append(path_counter)
                path_counter+=1
            self.each_user_pair_dafualt_paths[pair] = first_user_pair_path_ids
    def get_paths_between_user_pairs(self,user_pair,cut_off_for_path_searching):
        #print("for user pair ",user_pair,self.user_pairs)
        return self.k_shortest_paths(user_pair[0], user_pair[1], cut_off_for_path_searching,"weight")
    def k_shortest_paths(self,source, target, k, weight):
        return list(
            islice(nx.shortest_simple_paths(self.G, source, target, weight=weight), k)
        )

    
    def compute_e2e_fidleity(self,path_edges):
        edge_F = 0.99# we assume all links have this fidelity
        if path_edges:
            F_product = (4*edge_F-1)/3 
            for edge in path_edges[1:]:
                F_product  = F_product*(((4*edge_F)-1)/3)
        else:
            print("Error")
            return 1.0
        N = len(path_edges)
        p1 = 1
        p2 = 1
        F_final = (1/4)+(3/4)*((p1*(((4*(p2**2))-1)/3))**(N-1))*(F_product)

        return round(F_final,3)
    def compute_equation(self,e2e_rate,e2e_F):
        return math.log(e2e_rate*(e2e_F-0.5))

    def f(self,W,P,Q,i,k):
       
        if k==1:
            return Q[i][k]
        else:
            sum_Q = 0
            for l in range(i,W+1):
                sum_Q = sum_Q+Q[l][k]
            sum_P = 0
            for l in range(i+1,W+1):
                sum_P = sum_P+self.f(W,P,Q,l,k-1)
            return self.f(W,P,Q,i,k-1)* sum_Q+Q[i][k]* sum_P

    def compute_e2e_rate(self,path_id,path,W):

        scheme_key = str(W)+"-path"
        h =len(path)
        P={}
        Q={}
        q_value = 1.0
        self.p={}
        each_scheme_each_point_value = {}
        edge_indx = 1

        for edge in path:

            p_value = self.transmission[edge]
            self.p[edge_indx]=p_value
            edge_indx+=1
        for i in range(1,W+1):
            for k in range(1,h+1):
                try:
                    Q[i][k] =math.comb(W, i)*self.p[k]**i * (1-self.p[k])**(W-i)
                except:
                    Q[i]={}
                    Q[i][k] =math.comb(W, i)*self.p[k]**i * (1-self.p[k])**(W-i)

                if k==1:
                    try:
                        P[i][1] = Q[i][1]
                    except:
                        P[i]={}
                        P[i][1] = Q[i][1]

        for i in range(1,W+1):
            for k in range(1,h+1):
                value = self.f(W,P,Q,i,k)
                P[i][k] = value
        sum_i = 0
        for i in range(1,W+1):
            sum_i = sum_i+(i * P[i][k])
        EXT = q_value**(h-1) *sum_i


    #     print("The value of EXT for path %s is %s "%(path_id,EXT))
        return EXT

    

    def check_path_uses_repeater(self,u,p):

        
        for edge in self.set_of_paths[p]:
            if u in edge:
    #             print("path using repeater",edge)
                return True
        return False



    def compute_utility(self,path_id):


        if self.each_path_e_value[path_id]>0:
            e_value = self.each_path_e_value[path_id]
            e2e_F_value =self.each_path_e2e_F_value[path_id]
            utility_on_this_path = self.compute_equation(e_value,e2e_F_value)

        else:
            return self.utility_default_value
        return utility_on_this_path

    def maximizing_utility(self):
        """we solve the utility maximization using K repeaters and M memories"""

        opt_model = cpx.Model(name="maximizing_utility")
        x_p_vars  = {(p): opt_model.integer_var(lb=0, ub= 1,
                                  name="x_p_{0}".format(p))  
                   for user_pair in self.user_pairs
                for p in self.each_user_pair_paths[user_pair]}

        r_u_vars  = {(r): opt_model.integer_var(lb=0, ub= 1,
                                  name="r_{0}".format(r))  
                   for r in self.repeater_places}


        # Repeater memory constraint
        for u in self.repeater_places:
            opt_model.add_constraint(
                opt_model.sum(x_p_vars[p]*self.each_path_width[p]
                for user_pair in self.user_pairs
                for p in self.each_user_pair_paths[user_pair]
                if self.check_path_uses_repeater(u,p)
                )
                 <= self.D *r_u_vars[u], ctname="repeater_memory_constraint_{0}".format(u))

        # number of memory and repeaters constraint
        opt_model.add_constraint(
                opt_model.sum(r_u_vars[node]
                for node in self.repeater_places
                )<= min(self.M/self.D,self.K), ctname="number_of_repeaters_constraint")



        # All memory constraint
        opt_model.add_constraint(
            opt_model.sum(x_p_vars[p]*self.each_path_width[p]
            for user_pair in self.user_pairs
            for p in self.each_user_pair_paths[user_pair]
            if self.check_path_uses_repeater(u,p)
            )
             <= self.M, ctname="memory_constraint")

        for user_pair in self.user_pairs:
            opt_model.add_constraint(
                opt_model.sum(x_p_vars[p]
                    for p in self.each_user_pair_paths[user_pair]
                )
                     <= self.num_of_allowed_paths, ctname="using_one_path_constraint_{0}".format(user_pair))
    #     opt_model.add_constraint(
    #             opt_model.sum(x_p_vars[1]
    #             )
    #                  == 1, ctname="using_one_path_constraint_fixed1")


    #     opt_model.add_constraint(
    #             opt_model.sum(x_p_vars[405]
    #             )
    #                  == 1, ctname="using_one_path_constraint_fixed2")

        objective = opt_model.sum(self.compute_utility(p)*x_p_vars[0]
                              for pair in self.user_pairs for p in self.each_user_pair_paths[pair] 
                              )
        # for maximization
        opt_model.maximize(objective)
    #     opt_model.solve()
    #     opt_model.print_information()
        #try:
        opt_model.solve()
    #     opt_model.print_information()
    #     print('docplex.mp.solution',opt_model.solution)
    #     objective_value = utility_default_value
        set_of_used_repeaters = []
        set_of_used_paths = []


        try:
            if opt_model.solution:
                objective_value =opt_model.solution.get_objective_value()
        except ValueError:
            print(ValueError)
        nodes_string = ""

        if objective_value!=self.utility_default_value:

            used_nodes = []
            for user_pair in self.user_pairs:
                for p in self.each_user_pair_paths[user_pair]:
                    if x_p_vars[p].solution_value==1:
                        for edge in self.set_of_paths[p]:
                            if edge[0] not in used_nodes:
                                used_nodes.append(edge[0])
                                if nodes_string:
                                    nodes_string =nodes_string+":"+str(edge[0])
                                else:
                                    nodes_string = str(edge[0])
                            if edge[1] not in used_nodes:
                                used_nodes.append(edge[1])
                                if nodes_string:
                                    nodes_string =nodes_string+":"+str(edge[1])
                                else:
                                    nodes_string =str(edge[1])


        return objective_value,nodes_string
    
    def reset_all_variables(self):
        self.weights = dict()
        self.transmission = dict()
        self.each_path_width = {}
        self.each_repeater_memory = {}
        self.utility_default_value = -100
        self.p={}
        self.repeater_places = []
        self.set_of_paths_default = {}
        self.each_user_pair_dafualt_paths ={}
        self.each_path_e_value ={}
        self.each_path_e2e_F_value={}
        self.each_path_width = {}
        self.path_counter = 0
        self.set_of_paths = {}
        self.multi_path_state = False
        self.set_of_edges = []
        self.num_of_allowed_paths = 1
    def reset_path_variables(self):
        self.set_of_paths_default = {}
        self.each_user_pair_dafualt_paths ={}
        self.each_path_e_value ={}
        self.each_path_e2e_F_value={}
        self.each_path_width = {}
        self.path_counter = 0
        self.set_of_paths = {}
        
    def save(self,round_number,max_utility,used_repeaters,processing_time):
        with open("results/utility_maximization_results_final.csv", 'a') as newFile:                                
            newFileWriter = csv.writer(newFile)
            newFileWriter.writerow([round_number,self.N,self.square_dim_in_km,self.number_of_paths,
                                    self.num_of_allowed_paths,self.D,self.M,self.K,max_utility,
                                    used_repeaters,processing_time])

In [3]:

p={}
set_of_paths_default = {}
utility_values = set([])
each_user_pair_dafualt_paths ={}
each_path_e_value ={}
each_path_e2e_F_value={}
each_path_width = {}
path_counter = 0
set_of_paths = {}


    
def maximize_utility(set_of_number_of_nodes,square_dim_in_km_values,number_of_paths_values,M_values,D_values,K_values,multi_path_states):
    utility_default_value = -100
    network = Network()
    for round_number in range(100):
        for number_of_nodes in set_of_number_of_nodes:
            network.reset_all_variables()
            network.N = number_of_nodes
            for square_dim_in_km in square_dim_in_km_values:
                network.square_dim_in_km = square_dim_in_km
                network.generate_random_graph()
                
                for cut_off in number_of_paths_values:
                    network.reset_path_variables()
                    network.number_of_paths = cut_off
                    start_time = time.time()
                    network.compute_paths_between_pairs()
                    for num_of_allowed_paths in num_of_allowed_paths_values:
                        network.num_of_allowed_paths = num_of_allowed_paths
                        
                        """we enumerate all paths and compute their versions based on the value of W"""
                        for M in M_values:
                            network.M = M
                            for D in D_values:
                                network.D  = D
#                                 print("for D %s M %s "%(D,M))
                                network.set_of_paths = {}
                                network.each_user_pair_paths = {}
                                network.each_path_e_value  ={}
                                network.each_path_e2e_F_value = {}
                                path_id = 0
                                for user_pair,path_ids in network.each_user_pair_dafualt_paths.items():
#                                     print("we have %s paths for user pair %s "%(len(path_ids),user_pair))
                                    set_of_path_ids = []
                                    path_counter = 0
                                    for old_path_id  in path_ids:
                                        path_edges = network.set_of_paths_default[old_path_id]
                                        for W in range(0,M+1):
                                            E_p =network.compute_e2e_rate(path_id,path_edges,W)
                                            network.each_path_e_value[path_id] =E_p
                                            e2e_F = network.compute_e2e_fidleity(path_edges)
                                            network.each_path_e2e_F_value[path_id] = e2e_F
                                            network.each_path_width[path_id] = W
                                            network.set_of_paths[path_id] = path_edges
                                            set_of_path_ids.append(path_id)
                                            utility = network.compute_utility(path_id)
                        #                     print("user pair %s utility on %s th path %s nodes %s with D %s M %s W %s with rate %s e2e_F %s is %s "%(user_pair,path_counter,path_id,(len(path_edges)+1),D,M,W,E_p,e2e_F,utility))
                        #                     time.sleep(0.5)
                                            path_id+=1
#                                         print("E and e2e_F for all versions of path old path %s computed "%(old_path_id))
                                        path_counter+=1
                                    network.each_user_pair_paths[user_pair] = set_of_path_ids
                        #         print("finshed")
                                for K in K_values:
#                                     print("for K %s we have "%(K))
                                    network.K = K
                                    #user_pairs = [(2,8),(5,3)]
                                    utility,used_repeaters= network.maximizing_utility()
                                    if utility<-200:
                                        utility = -200
                                    current_time = time.time()
                                    procssing_time = int(current_time -start_time)
#                                     print("for D %s K %s M %s utility %s with using %s repeaters computed in %s seconds "%(D,K,M,utility,used_repeaters,procssing_time))
                        #             time.sleep(10)
                                    network.save(round_number,utility,used_repeaters,procssing_time)
                                print("round %s N %s L0 %s cut_off %s allowed %s M %s  M_values %s D %s from %s "%(round_number,number_of_nodes,square_dim_in_km,cut_off,num_of_allowed_paths,M,M_values,D,D_values))
                            print("round %s M %s  M_values %s "%(round_number,M,  M_values))
                        print("round_number %s number of allowed paths %s finished all %s "%(round_number, num_of_allowed_paths,num_of_allowed_paths_values))
                print("round_number %s square_dim_in_km %s from %s  "%(round_number, square_dim_in_km , square_dim_in_km_values))
            print("round_number %s number_of_nodes %s finished from %s "%(round_number,number_of_nodes in set_of_number_of_nodes))
        print("round %s finished from %s "%(round_number ,100))
M_values = [5,10,50,100,200,300,400]
D_values= [10,20,30,40,50,100]
K_values = [4,5,6,7,8,9,10,11,12,15]
set_of_number_of_nodes = [10,20,30,40]
square_dim_in_km_values = [100,200,300]
number_of_paths_values = [2,4,6]
num_of_allowed_paths_values = [1,2,3,4,5]

maximize_utility(set_of_number_of_nodes,
                 square_dim_in_km_values,number_of_paths_values,
                 M_values,D_values,K_values,
                 num_of_allowed_paths_values)


round 0 N 10 L0 100 cut_off 2 allowed 1 M 5  M_values [5, 10, 50, 100, 200, 300, 400] D 10 from [10, 20, 30, 40, 50, 100] 
round 0 N 10 L0 100 cut_off 2 allowed 1 M 5  M_values [5, 10, 50, 100, 200, 300, 400] D 20 from [10, 20, 30, 40, 50, 100] 
round 0 N 10 L0 100 cut_off 2 allowed 1 M 5  M_values [5, 10, 50, 100, 200, 300, 400] D 30 from [10, 20, 30, 40, 50, 100] 
round 0 N 10 L0 100 cut_off 2 allowed 1 M 5  M_values [5, 10, 50, 100, 200, 300, 400] D 40 from [10, 20, 30, 40, 50, 100] 
round 0 N 10 L0 100 cut_off 2 allowed 1 M 5  M_values [5, 10, 50, 100, 200, 300, 400] D 50 from [10, 20, 30, 40, 50, 100] 
round 0 N 10 L0 100 cut_off 2 allowed 1 M 5  M_values [5, 10, 50, 100, 200, 300, 400] D 100 from [10, 20, 30, 40, 50, 100] 
round 0 M 5  M_values [5, 10, 50, 100, 200, 300, 400] 
round 0 N 10 L0 100 cut_off 2 allowed 1 M 10  M_values [5, 10, 50, 100, 200, 300, 400] D 10 from [10, 20, 30, 40, 50, 100] 
round 0 N 10 L0 100 cut_off 2 allowed 1 M 10  M_values [5, 10, 50, 100, 200, 300, 

DOcplexLimitsExceeded: **** Promotional version. Problem size limits (1000 vars, 1000 consts) exceeded, model has 1214 vars, 14 consts, CPLEX code=1016